## Import

In [17]:
import os 
from glob import glob

from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model

import sys
import pickle
import gzip

In [18]:
def init_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        except RuntimeError as e:
            print(e)

## Set hyper parameter

In [19]:
# Hyper Parameter 
batch_size = 64
data_height = 32 
data_width = 32
channel_n = 3
num_classes = 43

In [20]:
def data():
    with gzip.open('pickle/train_images.pickle', 'rb') as f:
        train_images = pickle.load(f)

    with gzip.open('pickle/train_labels.pickle', 'rb') as f:
        train_labels = pickle.load(f)

    with gzip.open('pickle/test_images.pickle', 'rb') as f:
        test_images = pickle.load(f)

    with gzip.open('pickle/test_labels.pickle', 'rb') as f:
        test_labels = pickle.load(f)


    return train_images, train_labels, test_images, test_labels

In [25]:
def create_model(train_images, train_labels, test_images, test_labels):
    model = models.Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(data_height, data_width, 3)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    
    model.add(Dense(43, activation='softmax'))
    
    model.compile(loss="sparse_categorical_crossentropy", optimizer='adam',
        metrics=["accuracy"])
    
    return model

In [26]:
init_gpu()

X_train, Y_train, X_test, Y_test = data()

model = create_model(X_train, Y_train, X_test, Y_test)

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 512)              

In [27]:
model.fit(X_train, Y_train, epochs=10, batch_size=64)

test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=2)
print(test_loss, test_acc)

Epoch 1/10


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_18/Conv2D}}]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_18/Conv2D}}]]
	 [[metrics_10/acc/Identity/_437]]
0 successful operations.
0 derived errors ignored.